<a href="https://colab.research.google.com/github/25b3nk/llm-projects/blob/main/exploring_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.9 MB/s eta 0:00:00


In [3]:

# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [4]:

# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline



tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity': 'B-PER', 'score': 0.99110633, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9967968, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [6]:
example = "I want cameras from India"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-LOC', 'score': 0.997338, 'index': 5, 'word': 'India', 'start': 20, 'end': 25}]


In [7]:
!pip install spacy nltk
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import spacy
import nltk
from typing import Dict, List

class NLPQueryParser:
    def __init__(self):
        # Load English language model
        self.nlp = spacy.load("en_core_web_sm")

    def extract_entities_and_verbs(self, query: str) -> Dict[str, List[str]]:
        """
        Extract named entities and verbs from user query

        Args:
            query (str): Natural language query

        Returns:
            Dict containing entities and verbs
        """
        # Process query using spaCy
        doc = self.nlp(query)

        # Extract entities
        entities = {
            "PERSON": [],
            "ORG": [],
            "GPE": [],  # Geopolitical entities
            "PRODUCT": []
        }

        for ent in doc.ents:
            if ent.label_ in entities:
                entities[ent.label_].append(ent.text)

        # Extract verbs
        verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]

        return {
            "entities": entities,
            "verbs": verbs
        }

    def classify_intent(self, verbs: List[str]) -> str:
        """
        Basic intent classification based on verbs

        Args:
            verbs (List[str]): List of verbs in query

        Returns:
            str: Classified intent
        """
        intent_mapping = {
            "get": "RETRIEVE",
            "create": "CREATE",
            "update": "UPDATE",
            "delete": "DELETE"
        }

        for verb in verbs:
            if verb in intent_mapping:
                return intent_mapping[verb]

        return "UNKNOWN"

# Example Usage
def main():
    query_parser = NLPQueryParser()

    # Test queries
    queries = [
        "Show me all users in New York working at Google",
        "Create a new project for marketing team",
        "Update John's contact information"
    ]

    for query in queries:
        result = query_parser.extract_entities_and_verbs(query)
        intent = query_parser.classify_intent(result['verbs'])

        print(f"Query: {query}")
        print("Entities:", result['entities'])
        print("Verbs:", result['verbs'])
        print("Intent:", intent)
        print("---")

In [9]:
main()

Query: Show me all users in New York working at Google
Entities: {'PERSON': [], 'ORG': ['Google'], 'GPE': ['New York'], 'PRODUCT': []}
Verbs: ['show', 'work']
Intent: UNKNOWN
---
Query: Create a new project for marketing team
Entities: {'PERSON': [], 'ORG': [], 'GPE': [], 'PRODUCT': []}
Verbs: ['create']
Intent: CREATE
---
Query: Update John's contact information
Entities: {'PERSON': ['John'], 'ORG': [], 'GPE': [], 'PRODUCT': []}
Verbs: ['update']
Intent: UPDATE
---
